Person and vehicle transformation 

In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Database connection parameters
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Gouthi321@',  # Replace with your actual password
    'database': 'crash_data'
}

# Connect to the database
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_info)
        cursor = connection.cursor()

        # Define the file path to your CSV file
        csv_file_path = '/Users/gouthamvemula/Downloads/Dataset/crashes.csv'  # Replace with the actual file path

        # Read and insert data in chunks
        chunksize = 15000  # Adjust chunksize as per your system's capability
        for chunk in pd.read_csv(csv_file_path, chunksize=chunksize, dtype=str):
            # Replace NaN with None and convert strings to floats for lat and lon
            chunk = chunk.where(pd.notna(chunk), None)
            chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
            chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')

            # Format date columns
            chunk['CRASH DATE'] = pd.to_datetime(chunk['CRASH DATE'], errors='coerce').dt.strftime('%Y-%m-%d')

            # Filter out invalid latitudes and longitudes
            chunk = chunk[(chunk['LATITUDE'].between(-90, 90)) & (chunk['LONGITUDE'].between(-180, 180))]

            # Prepare location data
            chunk['LOCATION'] = chunk.apply(lambda x: f'POINT({x["LONGITUDE"]} {x["LATITUDE"]})' if pd.notna(x['LONGITUDE']) and pd.notna(x['LATITUDE']) else None, axis=1)

            # Convert the DataFrame to a list of tuples
            data_tuples = [tuple(x) for x in chunk.to_numpy()]

            # Prepare the insert statement with ON DUPLICATE KEY UPDATE clause
            insert_query = """
            INSERT INTO Crashes (
                `CRASH DATE`, `CRASH TIME`, `BOROUGH`, `ZIP CODE`, `LATITUDE`, `LONGITUDE`,
                `LOCATION`, `ON STREET NAME`, `CROSS STREET NAME`, `OFF STREET NAME`,
                `NUMBER OF PERSONS INJURED`, `NUMBER OF PERSONS KILLED`,
                `NUMBER OF PEDESTRIANS INJURED`, `NUMBER OF PEDESTRIANS KILLED`,
                `NUMBER OF CYCLIST INJURED`, `NUMBER OF CYCLIST KILLED`,
                `NUMBER OF MOTORIST INJURED`, `NUMBER OF MOTORIST KILLED`,
                `CONTRIBUTING FACTOR VEHICLE 1`, `CONTRIBUTING FACTOR VEHICLE 2`,
                `CONTRIBUTING FACTOR VEHICLE 3`, `CONTRIBUTING FACTOR VEHICLE 4`,
                `CONTRIBUTING FACTOR VEHICLE 5`, `COLLISION_ID`,
                `VEHICLE TYPE CODE 1`, `VEHICLE TYPE CODE 2`, `VEHICLE TYPE CODE 3`,
                `VEHICLE TYPE CODE 4`, `VEHICLE TYPE CODE 5`
            ) VALUES (%s, %s, %s, %s, %s, %s, ST_GeomFromText(%s, 4326), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                `CRASH DATE` = VALUES(`CRASH DATE`),
                `CRASH TIME` = VALUES(`CRASH TIME`),
                `BOROUGH` = VALUES(`BOROUGH`),
                `ZIP CODE` = VALUES(`ZIP CODE`),
                `LATITUDE` = VALUES(`LATITUDE`),
                `LONGITUDE` = VALUES(`LONGITUDE`),
                `LOCATION` = VALUES(`LOCATION`),
                `ON STREET NAME` = VALUES(`ON STREET NAME`),
                `CROSS STREET NAME` = VALUES(`CROSS STREET NAME`),
                `OFF STREET NAME` = VALUES(`OFF STREET NAME`),
                `NUMBER OF PERSONS INJURED` = VALUES(`NUMBER OF PERSONS INJURED`),
                `NUMBER OF PERSONS KILLED` = VALUES(`NUMBER OF PERSONS KILLED`),
                `NUMBER OF PEDESTRIANS INJURED` = VALUES(`NUMBER OF PEDESTRIANS INJURED`),
                `NUMBER OF PEDESTRIANS KILLED` = VALUES(`NUMBER OF PEDESTRIANS KILLED`),
                `NUMBER OF CYCLIST INJURED` = VALUES(`NUMBER OF CYCLIST INJURED`),
                `NUMBER OF CYCLIST KILLED` = VALUES(`NUMBER OF CYCLIST KILLED`),
                `NUMBER OF MOTORIST INJURED` = VALUES(`NUMBER OF MOTORIST INJURED`),
                `NUMBER OF MOTORIST KILLED` = VALUES(`NUMBER OF MOTORIST KILLED`),
                `CONTRIBUTING FACTOR VEHICLE 1` = VALUES(`CONTRIBUTING FACTOR VEHICLE 1`),
                `CONTRIBUTING FACTOR VEHICLE 2` = VALUES(`CONTRIBUTING FACTOR VEHICLE 2`),
                `CONTRIBUTING FACTOR VEHICLE 3` = VALUES(`CONTRIBUTING FACTOR VEHICLE 3`),
                `CONTRIBUTING FACTOR VEHICLE 4` = VALUES(`CONTRIBUTING FACTOR VEHICLE 4`),
                `CONTRIBUTING FACTOR VEHICLE 5` = VALUES(`CONTRIBUTING FACTOR VEHICLE 5`),
                `VEHICLE TYPE CODE 1` = VALUES(`VEHICLE TYPE CODE 1`),
                `VEHICLE TYPE CODE 2` = VALUES(`VEHICLE TYPE CODE 2`),
                `VEHICLE TYPE CODE 3` = VALUES(`VEHICLE TYPE CODE 3`),
                `VEHICLE TYPE CODE 4` = VALUES(`VEHICLE TYPE CODE 4`),
                `VEHICLE TYPE CODE 5` = VALUES(`VEHICLE TYPE CODE 5`)
            """

            # Execute the insert query
            cursor.executemany(insert_query, data_tuples)
            connection.commit()
            print(f"Processed {cursor.rowcount} rows in the Crashes table.")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.3.0
Error while connecting to MySQL Failed executing the operation; Not enough parameters for the SQL statement
MySQL connection is closed


In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Database connection parameters
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Gouthi321@',  # Replace with your actual password
    'database': 'crash_data'
}

# Connect to the database
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_info)
        cursor = connection.cursor()

        # Define the file path to your CSV file
        csv_file_path = '/Users/gouthamvemula/Downloads/Dataset/crashes.csv'

        # Read and insert data in chunks
        chunksize = 15000  # Adjust chunksize as per your system's capability
        for chunk in pd.read_csv(csv_file_path, chunksize=chunksize, dtype=str):
            # Replace NaN with None and convert strings to floats for lat and lon
            chunk = chunk.where(pd.notna(chunk), None)
            chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
            chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')

            # Format date columns
            chunk['CRASH DATE'] = pd.to_datetime(chunk['CRASH DATE'], errors='coerce').dt.strftime('%Y-%m-%d')

            # Filter out invalid latitudes and longitudes
            chunk = chunk[(chunk['LATITUDE'].between(-90, 90)) & (chunk['LONGITUDE'].between(-180, 180))]

            # Prepare location data
            chunk['LOCATION'] = chunk.apply(lambda x: f'POINT({x["LONGITUDE"]} {x["LATITUDE"]})' if pd.notna(x['LONGITUDE']) and pd.notna(x['LATITUDE']) else None, axis=1)

            # Convert the DataFrame to a list of tuples
            columns = [
                'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
                'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
                'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
                'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
                'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
                'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
                'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
                'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
                'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID',
                'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
                'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'
            ]
            data_tuples = [tuple(row) for _, row in chunk[columns].iterrows()]

            # Prepare the insert statement with ON DUPLICATE KEY UPDATE clause
            placeholders = ', '.join(['%s'] * len(columns))
            insert_query = f"""
            INSERT INTO Crashes (
                `{'`, `'.join(columns)}`
            ) VALUES ({placeholders})
            ON DUPLICATE KEY UPDATE
                `{'` = VALUES(`'.join(columns)}`)
            """

            # Execute the insert query
            cursor.executemany(insert_query, data_tuples)
            connection.commit()
            print(f"Processed {cursor.rowcount} rows in the Crashes table.")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.3.0
MySQL connection is closed


KeyError: "['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5'] not in index"

crashes table insert statement

In [5]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Database connection parameters
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Gouthi321@',  # Replace with your actual password
    'database': 'crash_data'
}

# Connect to the database
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_info)
        cursor = connection.cursor()

        #  file path to your CSV file
        csv_file_path = '/Users/gouthamvemula/Downloads/Motor_Vehicle_Collisions_-_Crashes_20240425 (2).csv'  # Replace with the actual file path

        # Read and insert data in chunks
        chunksize = 15000  # Adjust chunksize as per your system's capability
        for chunk in pd.read_csv(csv_file_path, chunksize=chunksize, dtype=str):
            # Replace NaN with None and convert strings to floats for lat and lon
            chunk = chunk.where(pd.notna(chunk), None)
            chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
            chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')

            # Format date columns
            chunk['CRASH DATE'] = pd.to_datetime(chunk['CRASH DATE'], errors='coerce').dt.strftime('%Y-%m-%d')

            # Filter out invalid latitudes and longitudes
            chunk = chunk[(chunk['LATITUDE'].between(-90, 90)) & (chunk['LONGITUDE'].between(-180, 180))]

            # Prepare location data
            chunk['LOCATION'] = chunk.apply(lambda x: f'POINT({x["LONGITUDE"]} {x["LATITUDE"]})' if pd.notna(x['LONGITUDE']) and pd.notna(x['LATITUDE']) else None, axis=1)

            # Convert the DataFrame to a list of tuples
            data_tuples = [tuple(x) for x in chunk.to_numpy()]

            # Prepare the insert statement based on the table columns
            insert_query = """
            INSERT INTO Crashes (
                `CRASH DATE`, `CRASH TIME`, `BOROUGH`, `ZIP CODE`, `LATITUDE`, `LONGITUDE`,
                `LOCATION`, `ON STREET NAME`, `CROSS STREET NAME`, `OFF STREET NAME`, 
                `NUMBER OF PERSONS INJURED`, `NUMBER OF PERSONS KILLED`,
                `NUMBER OF PEDESTRIANS INJURED`, `NUMBER OF PEDESTRIANS KILLED`, 
                `NUMBER OF CYCLIST INJURED`, `NUMBER OF CYCLIST KILLED`, 
                `NUMBER OF MOTORIST INJURED`, `NUMBER OF MOTORIST KILLED`,
                `CONTRIBUTING FACTOR VEHICLE 1`, `CONTRIBUTING FACTOR VEHICLE 2`, 
                `CONTRIBUTING FACTOR VEHICLE 3`, `CONTRIBUTING FACTOR VEHICLE 4`, 
                `CONTRIBUTING FACTOR VEHICLE 5`, `COLLISION_ID`,
                `VEHICLE TYPE CODE 1`, `VEHICLE TYPE CODE 2`, `VEHICLE TYPE CODE 3`, 
                `VEHICLE TYPE CODE 4`, `VEHICLE TYPE CODE 5`
            ) VALUES (%s, %s, %s, %s, %s, %s, ST_GeomFromText(%s, 4326), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            # Execute the insert query
            cursor.executemany(insert_query, data_tuples)
            connection.commit()
            print(f"Inserted {cursor.rowcount} rows into Crashes table.")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Connected to MySQL Server version  8.3.0
Error while connecting to MySQL 1054 (42S22): Unknown column 'CRASH DATE' in 'field list'
MySQL connection is closed
